In [1]:
import h5py
import numpy as np
import pandas as pd
import Trial_Classifier as TC

import Classification_Utils as CU

In [4]:
# load data and update preprocessor object
kin_filename = f'{TC.folder_name}/kin_rm16_9_17_s2.pkl'
exp_filename = f'{TC.folder_name}/exp_rm16_9_17_s2.pkl'
label = CU.rm16_9_17_s2_label
vec_label, _ = CU.make_vectorized_labels(label)
preprocessor = TC.Preprocessor()
kin_block = TC.Preprocessor.load_data(kin_filename, file_type='pkl')
exp_block = TC.Preprocessor.load_data(exp_filename, file_type='pkl')

# trialize
trials, times = TC.Preprocessor.split_trial(kin_block, exp_block, 250, 10)

# take example trial
trial = trials[0]
time = times[0]

#display(trial)
#display(time)

trial_arr = trial.values # convert df to array where rows are frame numbers, cols are bodyparts
trial_arr = trial_arr.T # array with rows are XYZ bodyparts, cols are frame nums

# display(t)

### Calc Pos, Vel, Speed ###

In [65]:
class MakeFeatures:
    # Operates on a single trial.
    
    pos_names = ['Handle', 'Back Handle', 'Nose',
                 'Left Shoulder', 'Left Forearm', 'Left Wrist', 'Left Palm', 'Left Index Base', 'Left Index Tip',
                 'Left Middle Base', 'Left Middle Tip', 'Left Third Base',
                 'Left Third Tip', 'Left Fourth Finger Base', 'Left Fourth Finger Tip',
                 'Right Shoulder', 'Right Forearm', 'Right Wrist', 'Right Palm', 'Right Index Base',
                 'Right Index Tip', 'Right Middle Base', 'Right Middle Tip', 'Right Third Base',
                 'Right Third Tip', 'Right Fourth Finger Base', 'Right Fourth Finger Tip']
    
    def __init__(self, trial_arr):
        # partition coords and probabilities
        self.num_bodyparts = 27 
        self.num_coords = 3 
        self.split_index = self.num_bodyparts * self.num_coords # 27 bodyparts * 3 XYZ coordinates for each = 81
        self.coords = trial_arr[:self.split_index] # all XYZ coords of all bodyparts (81 rows of first half of array)
        self.prob = trial_arr[self.split_index:] # all probability columns (81 rows of second half of array)
        
        #display(coords, prob)

    def calc_position(self):
        # calculate position of each bodypart (x+y+z/3)
        positions = []  # 2D array with rows are bodyparts, cols are frame nums
        for i in np.arange(0, len(self.coords), self.num_coords):  # for every bodypart
            X = self.coords[i]
            Y = self.coords[i+1]
            Z = self.coords[i+2]
            pos = (X+Y+Z)/self.num_coords # 1D array
            positions.append(pos) 

        assert(len(positions) == self.num_bodyparts)
        return positions 

    def calc_velocity_speed(self, time):
        """
        Time is sliced from exp block 'time' column
        """
        # calculate velocity for each XYZ bodypart (x1-x0/t0-t1) 
        velocities = []  # 2D array with rows are XYZ bodyparts, cols are frame nums
        for i in np.arange(0, self.split_index, self.num_coords):  # for every bodypart
            X = self.coords[i]
            Y = self.coords[i+1]
            Z = self.coords[i+2]

            for arr in [X, Y, Z]: 
                vel = []
                for j in np.arange(len(arr)-1):
                    x_0 = arr[j]
                    x_1 = arr[j+1]
                    t_0 = time[j]
                    t_1 = time[j+1]
                    vel.append(x_1-x_0/t_1-t_0)
                velocities.append(vel)

        assert(len(velocities) == self.split_index)

        # calculate speed of each bodypart (vel_x+vel_y+vel_z/3)
        speeds = [] # 1D array with rows are bodyparts, cols are frame nums
        for i in np.arange(0, self.split_index, self.num_coords):
            x_vel = velocities[i]
            y_vel = velocities[i+1]
            z_vel = velocities[i+2]

            x_squared = np.dot(x_vel, x_vel)
            y_squared = np.dot(y_vel, y_vel)
            z_squared = np.dot(z_vel, z_vel)

            speed = (x_squared+y_squared+z_squared) / 3  # int
            speeds.append(speed)

        assert(len(speeds) == self.num_bodyparts)
        return velocities, speeds

    @staticmethod
    def calc_all(trial_arr, time):
        # Calculate
        f = MakeFeatures(trial_arr)
        positions = f.calc_position()
        velocities, speeds = f.calc_velocity_speed(time)

        # take mean & median of each bodypart for 2D arrays
        mean_vel = np.mean(velocities, axis=1) # len = 81
        median_vel = np.median(velocities, axis=1)

        mean_pos = np.mean(positions, axis=1)  # len = 27
        median_pos = np.median(positions, axis=1)

        # Create df
        # concat all arrays
        speeds.extend(mean_pos)
        speeds.extend(median_pos)
        speeds.extend(mean_vel)
        speeds.extend(median_vel)
        # create col names
        col_names = [bodypart+' speed' for bodypart in f.pos_names]
        col_names.extend([bodypart+' mean pos' for bodypart in f.pos_names])
        col_names.extend([bodypart+' median pos' for bodypart in f.pos_names])
        xzy_pos_names = [bodypart+' X' for bodypart in f.pos_names] + [bodypart+' Y' for bodypart in f.pos_names] + [bodypart+' Z' for bodypart in f.pos_names] 
        col_names.extend([bodypart+' mean vel' for bodypart in xzy_pos_names])
        col_names.extend([bodypart+' median vel' for bodypart in xzy_pos_names])
        # create df
        df = pd.DataFrame([speeds], columns=col_names)
        return df


# Make Block Features
df = pd.DataFrame()
for i in range(len(trials)):
    # take trial
    trial = trials[i]
    time = times[i]

    trial_arr = trial.values # convert df to array where rows are frame numbers, cols are bodyparts
    trial_arr = trial_arr.T # array with rows are XYZ bodyparts, cols are frame nums
    
    df = pd.concat([df, MakeFeatures.calc_all(trial_arr, time)])
df.reset_index(drop=True, inplace=True)  

# rows are trials, cols are features
df

#save
save_as = f'{TC.folder_name}/features0'
preprocessor.save_data(df, save_as, file_type='pkl')

In [67]:
# load
preprocessor.load_data(f'{TC.folder_name}/features0')

,Handle speed,Back Handle speed,Nose speed,Left Shoulder speed,Left Forearm speed,Left Wrist speed,Left Palm speed,Left Index Base speed,Left Index Tip speed,Left Middle Base speed,...,Right Wrist Z median vel,Right Palm Z median vel,Right Index Base Z median vel,Right Index Tip Z median vel,Right Middle Base Z median vel,Right Middle Tip Z median vel,Right Third Base Z median vel,Right Third Tip Z median vel,Right Fourth Finger Base Z median vel,Right Fourth Finger Tip Z median vel
0,1.792911e+06,1.792971e+06,1.792504e+06,1.792634e+06,1.792565e+06,1.792631e+06,1.792788e+06,1.792584e+06,1.792694e+06,1.792864e+06,...,-83.010209,-83.031459,-83.044451,-83.008406,-83.021352,-83.042663,-83.013812,-83.028465,-83.042066,-83.016268
1,2.524606e+06,2.524661e+06,2.524024e+06,2.524465e+06,2.524611e+06,2.524578e+06,2.524597e+06,2.524531e+06,2.524429e+06,2.524699e+06,...,-98.568082,-98.495289,-98.582087,-98.582156,-98.494954,-98.570323,-98.561753,-98.492525,-98.577841,-98.569413
2,3.444620e+06,3.444602e+06,3.443826e+06,3.444497e+06,3.445111e+06,3.444938e+06,3.445176e+06,3.444416e+06,3.444792e+06,3.444873e+06,...,-115.093842,-115.101299,-115.119361,-115.109278,-115.094735,-115.118965,-115.102227,-115.097014,-115.120495,-115.112453
3,3.915876e+06,3.915945e+06,3.915229e+06,3.915613e+06,3.915961e+06,3.916414e+06,3.916730e+06,3.914981e+06,3.916246e+06,3.915417e+06,...,-122.719686,-122.716381,-122.736227,-122.723057,-122.706765,-122.739138,-122.726145,-122.712332,-122.739379,-122.728434
4,4.429009e+06,4.429051e+06,4.428382e+06,4.428683e+06,4.429312e+06,4.429421e+06,4.429665e+06,4.427964e+06,4.429185e+06,4.427687e+06,...,-130.500479,-130.518032,-130.523712,-130.502997,-130.507823,-130.521550,-130.506068,-130.513181,-130.524442,-130.505842
5,5.031025e+06,5.031030e+06,5.030138e+06,5.030125e+06,5.030806e+06,5.031216e+06,5.031668e+06,5.029286e+06,5.031393e+06,5.029492e+06,...,-139.086980,-139.091481,-139.112487,-139.101336,-139.084516,-139.112629,-139.096221,-139.087834,-139.115070,-139.104942
6,6.474498e+06,6.474518e+06,6.473432e+06,6.473910e+06,6.474765e+06,6.474788e+06,6.475039e+06,6.474659e+06,6.474486e+06,6.474935e+06,...,-157.784945,-157.799901,-157.811778,-157.796302,-157.792517,-157.812863,-157.795191,-157.795555,-157.814380,-157.800722
7,7.265588e+06,7.265610e+06,7.264622e+06,7.265118e+06,7.265692e+06,7.265873e+06,7.266313e+06,7.264297e+06,7.265473e+06,7.266675e+06,...,-167.160299,-167.163120,-167.179053,-167.166408,-167.154708,-167.179201,-167.165921,-167.158977,-167.181151,-167.167228
8,9.648996e+06,9.649094e+06,9.647980e+06,9.648523e+06,9.648966e+06,9.649068e+06,9.649149e+06,9.648682e+06,9.648729e+06,9.649047e+06,...,-192.657140,-192.596245,-192.677374,-192.670439,-192.593292,-192.671130,-192.652412,-192.594003,-192.676221,-192.656305
9,1.033539e+07,1.033548e+07,1.033463e+07,1.033503e+07,1.033547e+07,1.033561e+07,1.033568e+07,1.033549e+07,1.033572e+07,1.033583e+07,...,-199.403416,-199.335982,-199.431850,-199.421268,-199.338810,-199.414549,-199.392182,-199.336307,-199.420737,-199.397236
